# Web Scraping para la Recopilación de Datos de Aeropuertos en América

## Introducción:

Este Jupyter Notebook tiene como objetivo principal la recopilación de datos de aeropuertos en América mediante técnicas de web scraping. En el contexto de nuestro proyecto de base de datos orientada a grafos para la gestión de aeropuertos, es esencial contar con información actualizada y precisa sobre los aeropuertos en la región.

El web scraping nos permitirá extraer datos relevantes de fuentes en línea, como sitios web de autoridades de aviación y aerolíneas. Estos datos incluirán detalles sobre ubicaciones geográficas, nombres de aeropuertos, servicios disponibles, rutas operadas y otros atributos esenciales.

## Objetivos:

Obtener datos de aeropuertos en América a partir de fuentes en línea.
Extraer información clave, como nombres de aeropuertos, ubicaciones y detalles operativos.
Almacenar los datos recopilados en un formato adecuado para su posterior análisis y uso en nuestra base de datos orientada a grafos.

### Autor

Este Python Notebook está hecho por **Manrique Camacho P**

## Importación de librerias

* **Requests**: La biblioteca requests le permite al usuario realizar solicitudes HTTP a las páginas web que desee analizar, facilitando la descarga del contenido HTML de dichas páginas para su posterior procesamiento.

* **Beautiful Soup (bs4)**: Beautiful Soup es una herramienta útil para analizar y buscar elementos HTML en el contenido descargado. Permite al usuario buscar y extraer información específica de las páginas web, como títulos, párrafos, enlaces y más.

* **Selenium**: Cuando los sitios web utilizan JavaScript para cargar contenido dinámico, selenium se convierte en una opción valiosa. Con esta biblioteca, el usuario puede automatizar un navegador web para interactuar con el sitio web y extraer datos de páginas que requieren interacción.

* **Pandas**: Pandas es una biblioteca esencial para estructurar y manipular los datos extraídos. Permite al usuario crear DataFrames para organizar los datos en filas y columnas, lo que facilita las operaciones de limpieza, filtrado y procesamiento.

In [1]:
import requests
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import ui
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

## Web Scraping para Recopilación de Datos

En esta sección, se explorará el web scraping, una técnica automatizada para obtener datos de sitios web. Los temas clave incluyen:

* **Solicitudes HTTP**: Se utilizará la biblioteca requests para descargar el contenido HTML de las páginas web.
* **Análisis HTML**: Se empleará Selenium para interactuar con sitios web dinámicos y extraer información relevante.
* **Recopilación de Datos**: Se llevará a cabo la recolección de datos desde los sitios web seleccionados.

In [2]:
chromedriver_autoinstaller.install()  # Instala automáticamente la versión correcta de ChromeDriver

'/Users/manriquecamacho/anaconda3/lib/python3.11/site-packages/chromedriver_autoinstaller/118/chromedriver'

In [3]:
aeropuertos_links = ["arrivals/ord/airport-chicago-o'hare","departures/ord/airport-chicago-o'hare", #Chicago, USA
                    "arrivals/sal/airport-san-salvador-cuscatlan","departures/sal/airport-san-salvador-cuscatlan", #El Salvador
                    "arrivals/pac/airport-panama-city-albrook-marcos-a.-gelabert","departures/pac/airport-panama-city-albrook-marcos-a.-gelabert", #Panamá
                    "arrivals/gua/airport-guatemala-city-la-aurora", "departures/gua/airport-guatemala-city-la-aurora", #Guatemala
                    "arrivals/pur/airport-puerto-rico","departures/pur/airport-puerto-rico", #Puerto Rico
                    "arrivals/tza/airport-belize-city-municipal","departures/tza/airport-belize-city-municipal", #Belize
                    "arrivals/cap/airport-cap-haitien-cap-haitien", "departures/cap/airport-cap-haitien-cap-haitien", #Haití
                    "arrivals/mvd/airport-montevideo-carrasco", "departures/mvd/airport-montevideo-carrasco", #Uruguay
                    "arrivals/pse/airport-ponce-mercedita", "departures/pse/airport-ponce-mercedita", #Puerto Rico
                    "arrivals/sju/airport-san-juan-luis-munoz-marin","departures/sju/airport-san-juan-luis-munoz-marin", #Puerto Rico
                    "arrivals/sdq/airport-santo-domingo-las-americas", "departures/sdq/airport-santo-domingo-las-americas", #Dominicana
                    "arrivals/mga/airport-managua", "departures/mga/airport-managua", #Nicaragua
                    "arrivals/sap/airport-san-pedro-sula-ramon-villeda-morales", "departures/sap/airport-san-pedro-sula-ramon-villeda-morales", #Honduras
                    "arrivals/tgu/airport-tegucigalpa", "departures/tgu/airport-tegucigalpa", #Honduras 
                    "arrivals/iah/airport-houston-george-bush-intercontinental","departures/iah/airport-houston-george-bush-intercontinental", #Houston, USA
                    "arrivals/mco/airport-orlando","departures/mco/airport-orlando", #Orlando, USA
                    "arrivals/mex/airport-mexico-city-benito-juarez","departures/mex/airport-mexico-city-benito-juarez", #Mexico
                    "arrivals/jfk/airport-new-york-john-f.-kennedy","departures/jfk/airport-new-york-john-f.-kennedy", #NY,USA
                    "arrivals/eoh/airport-medellin-enrique-olaya-herrera","arrivals/eoh/airport-medellin-enrique-olaya-herrera", #Medellin, Colombia
                    "arrivals/srs/airport-san-marcos-cartagena","departures/srs/airport-san-marcos-cartagena",#Cartagena, Colombia
                    "arrivals/scl/airport-santiago-comodoro-arturo-merino-benitez","departures/scl/airport-santiago-comodoro-arturo-merino-benitez",#Santiago, Chile
                    "arrivals/dfw/airport-dallas-fort-worth","departures/dfw/airport-dallas-fort-worth", #Dallas,USA
                    "arrivals/sfo/airport-san-francisco","departures/sfo/airport-san-francisco", #San Francisco, USA
                    "arrivals/clt/airport-charlotte-douglas","departures/clt/airport-charlotte-douglas", #Charlotte, USA
                    "arrivals/phx/airport-phoenix-sky-harbor", "departures/phx/airport-phoenix-sky-harbor", #Phoenix, USA
                    "arrivals/mia/airport-miami", "departures/mia/airport-miami", #Miami, USA
                    "arrivals/phl/airport-new-philadelphia-philadelphia", "departures/phl/airport-new-philadelphia-philadelphia", #Philadelphia, USA
                    "arrivals/yvr/airport-richmond-vancouver", "departures/yvr/airport-richmond-vancouver", #Vancouver, Canadá
                    "arrivals/cun/airport-cancun", "departures/cun/airport-cancun", #Cancún, México
                    "arrivals/yyz/airport-toronto-pearson","departures/yyz/airport-toronto-pearson",#Toronto, Canadá  
                    "arrivals/bsb/airport-brasilia-distrito-federal-brasilia", "departures/bsb/airport-brasilia-distrito-federal-brasilia" #Brasilia, Brazil
                    ] 

In [4]:
def airport_basic(extension): #Nos dara la información general del aeropuerto
    
    res = requests.get(f"https://airportinfo.live/{extension}",headers = {'User-Agent': 'Mozilla/5.0'})
    
    soup = bs4.BeautifulSoup(res.text, "html.parser")

    nombre = soup.find_all('strong')[-1].get_text(strip=True) 
    telephone = ' '.join(soup.find_all('li')[-3].text.split()[2:])
    timezone = ' '.join(soup.find_all('li')[-2].text.split()[len(soup.find_all('li')[-2].text.split())-2:])

    data = []
    
    for i in soup.find_all('li')[-4]:
        
        data.append(i)
    
    location = data[2]
    
    info_ae = [nombre, location, timezone, telephone]
    
    return info_ae

In [5]:
def wb_driver(extension):
    
    driver = webdriver.Chrome()
    
    driver.get(f"https://airportinfo.live/{extension}")
    
    return driver

In [18]:
def recoleccion(extension):
    
    
    info = airport_basic(extension)[0].split()[0]
    
    driver = wb_driver(extension)
    
    data = []
    
    for i in range(1,25):
        
        hora = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="timeStart"]')))

        driver.execute_script("return arguments[0].scrollIntoView(true);", hora)

        hora_fija = driver.find_element(By.XPATH,f'//*[@id="timeStart"]/option[{i}]')

        hora_fija.click()

        boton_hora = driver.find_element(By.XPATH, '//*[@id="fs_div"]/div[3]/button')

        boton_hora.click()
        
        time.sleep(7)
        
    
        try:

            vuelos = driver.find_element(By.TAG_NAME, 'tbody')

            vuelos = vuelos.text.split('\n')
            
            a = 0
            
            j = 0
            
            while a != len(vuelos):
            
                arrivo = "NO"

                while arrivo not in ["SCHEDULED","ARRIVED","IN AIR",
                                     'You’re eligible for compensation from this flight. Check what you’re owed for free. Check now',
                                     "UNKNOWN"]:

                    arrivo = vuelos[a]

                    if arrivo in ["SCHEDULED","ARRIVED","IN AIR",
                                  "You’re eligible for compensation from this flight. Check what you’re owed for free. Check now",
                                  "UNKNOWN"]:

                        data.append(vuelos[j:(a+1)])

                        j = a+1

                    a +=1

                    if a == len(vuelos):

                        break


        except NoSuchElementException:

            pass
    
    return data

In [64]:
def get_airlines():
    
    # URL de la página de Wikipedia
    url = "https://en.wikipedia.org/wiki/List_of_airline_codes"

    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar la tabla que contiene los códigos IATA
    iata_table = soup.find('table', {'class': 'wikitable'})

    # Inicializar una lista para almacenar los datos de la tabla
    iata_data = []

    # Recorrer las filas de la tabla
    for row in iata_table.find_all('tr')[1:]:  # Ignorar la primera fila que contiene encabezados
        
        # Obtener todas las celdas de la fila
        cells = row.find_all('td')
        
        if len(cells) >= 4:
            
            # Obtener el código IATA y comprobar si no está vacío
            iata_code = cells[0].get_text(strip=True)
            iaco = cells[1].get_text(strip=True)
            
            if iata_code and iaco:
                
                # Si el código IATA ni IACO no está vacío
                airline_name = cells[2].get_text(strip=True)
                call_sign = cells[3].get_text(strip=True)
                country = cells[4].get_text(strip=True)

                # Agregar los datos a la lista
                iata_data.append([iata_code, iaco, airline_name, call_sign,country])
                
    return iata_data

## Creación de DataFrames

Los DataFrames son una estructura tabular proporcionada por la biblioteca de **pandas**, desempeñan un rol central al proporcionar una base sólida desde la cual se insertarán los datos en sistemas de bases de datos, tales como ArangoDB y OrientDB.

### Utilizando pandas para la Creación de DataFrames

La librería **pandas** ofrece un conjunto completo de herramientas y funciones diseñadas para la carga, manipulación y análisis de datos en formato tabular. Durante esta fase del proyecto, se empleará **pandas** para la creación de DataFrames que representarán una variedad de datos, incluyendo información sobre aeropuertos, rutas, aerolíneas y otros atributos relevantes.

La creación de DataFrames con **pandas** asegura una organización y estructura adecuada de los datos antes de que se inserten en bases de datos de grafos, como ArangoDB y OrientDB, que se utilizarán en etapas posteriores del proyecto.

### Preparación para la Inserción en Bases de Datos de Grafos

La creación de DataFrames ayuda a la gestión de datos de aeropuertos. Utilizando **pandas** para esta tarea, se facilita la transición de datos desde una estructura tabular hacia una representación basada en grafos, en ArangoDB como en OrientDB.

Se crearan dos bases de datos en formato csv
1. **Base de Datos de Aeropuertos**: Esta base contiene información general sobre los aeropuertos, como el nombre, ubicación geográfica, zona horaria y número de contacto.

3. **Base de Datos de las Aerolineas**: Esta base contiene información de las aerolíneas; el código IATA, IACO, nombre de la aerolínea, llamada de identifiación, país origen.

2. **Base de Datos de Vuelos**: Esta base se enfoca en detalles específicos de los vuelos, como las horas de salida y llegada, la aerolínea y otros atributos relacionados con los vuelos.

La organización de los DataFrames son importantes para garantizar la calidad de los datos antes de que sean insertados en las bases de datos de grafos de manera correcta.


In [9]:
#Base de Datos de Aeropuertos

df_info_aero = pd.DataFrame(columns=["Nombre","Ubicación","Timezone","Teléfono"],
                           index= [x for x in range(0,len(aeropuertos_links)-1,2)])

for aeropuerto in range(0,len(aeropuertos_links)-1,2):
    
    info = airport_basic(aeropuertos_links[aeropuerto])
    
    df_info_aero.loc[aeropuerto] = [info[0],info[1],info[2],info[3]]
    
    time.sleep(5)
    
df_info_aero.head()

In [12]:
##Exportación de Datos de Aeropuertos a formato csv

df_info_aero.to_csv("Info_Basica_Aeropuertos.csv")

In [73]:
#Base de Datos de las Aerolineas

aerolinea = get_airlines()

df_aerolineas = pd.DataFrame(aerolinea, columns=["IATA", "IACO", "Aerolinea","Llamada de Identificación", "País"])

df_aerolineas.tail(7)

,IATA,IACO,Aerolinea,Llamada de Identificación,País
1433,4Y,UYA,Yute Air Alaska,,United States
1434,2N,UMK,Yuzhmashavia,YUZMASH,Ukraine
1435,C4,IMX,Zimex Aviation,ZIMEX,Switzerland
1436,Q3,MBN,Zambian Airways,ZAMBIANA,Zambia
1437,3J,WZP,Zip,ZIPPER,Canada
1438,ZG,TZP,Zipair Tokyo,ZIPPY,Japan
1439,Z4,OOM,Zoom Airlines,ZOOM,Canada


In [74]:
## Exportación de Base de Datos de las Aerolineas

df_aerolineas.to_csv("Aerolineas.csv")

In [69]:
#Base de Datos de Vuelos 

data = recoleccion(aeropuertos_links[0])

,IATA,IACO,Aerolinea,Llamada de Identificación,País
0,PR,BOI,2GO,ABAIR,Philippines
1,1A,n/a,Amadeus IT Group S.A.,n/a,Global
2,1B,n/a,Sabre travel network Asia-Pacific (ex-Abacus),n/a,APAC
3,1E,n/a,Travelsky,n/a,China
4,1F,n/a,"Infini travel information, Inc.",n/a,Japan
...,...,...,...,...,...
1435,C4,IMX,Zimex Aviation,ZIMEX,Switzerland
1436,Q3,MBN,Zambian Airways,ZAMBIANA,Zambia
1437,3J,WZP,Zip,ZIPPER,Canada
1438,ZG,TZP,Zipair Tokyo,ZIPPY,Japan
